In [4]:
def image_show(image, nrows=1, ncols=1, cmap='gray'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 14))
    ax.imshow(image, cmap='gray')
    ax.axis('off')
    return fig, ax

In [4]:
#Watershed on combined image region

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('/home/rcardiff/combined_img-4.png')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray_inv = np.invert(gray)

kernel1 = np.array([[-1,-1, -1], [0, 0, 0], [1, 1, 1]], np.uint8)
opening = cv.morphologyEx(gray_inv,cv.MORPH_OPEN,kernel1, iterations = 3)
#opening = dilation. turns pixel to 1 if a pixel under the kernel is 1, increasing size of object and 
#helping connect broken objects

kernel2 = np.ones((3,3),np.uint8)
grad = cv.morphologyEx(opening,cv.MORPH_GRADIENT, kernel2, iterations =1)


# Marker labelling
ret, markers = cv.connectedComponents(grad)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
markers = cv.watershed(img,markers)
#image_show(-markers[10000:15000])

from skimage.measure import regionprops
print(len(regionprops(markers)))

196678


In [ ]:
#not this one
img = cv.imread('/home/rcardiff/combined_img-4.png')
image_show(img[4:7,189:192])
img2 = cv.imread('/home/rcardiff/high_res_test/neg_CL1.png')

region = regionprops(markers)[2]
print(region.bbox)
min_mz,min_rt,max_mz,max_rt = region.bbox
single_img_crop = img2[min_mz:max_mz,min_rt:max_rt]

image_show(single_img_crop)
image_show(img2[4:7,189:192])
print(img[5,190])

gray = cv.cvtColor(single_img_crop, cv.COLOR_BGR2GRAY)
gray_inv = np.invert(gray)
kernel = np.ones((3,3),np.uint8)
new_grad = cv.morphologyEx(gray_inv,cv.MORPH_GRADIENT, kernel, iterations =1)

ret, new_markers = cv.connectedComponents(new_grad)
new_markers = new_markers+1
new_markers = cv.watershed(single_img_crop,new_markers)

print(regionprops(new_markers)[0].bbox)
low_mz, low_rt, high_mz, high_rt = regionprops(new_markers)[0].bbox
image_show(new_markers)
med_mz = int((low_mz+high_mz)/2)
med_rt = int((low_rt+high_rt)/2)
print(single_img_crop[med_mz,med_rt])
print(new_markers[med_mz,med_rt])

In [103]:
#not this one
import matplotlib.patches as mpatches
import os

directory = "/home/rcardiff/high_res_test/"
os.chdir(directory)
files = os.listdir(directory)

c = 0
feature_list = {}

#for each feature (A) in combined image:
for region in regionprops(markers):
    min_mz, min_rt, max_mz, max_rt = region.bbox
    if max_rt - min_rt >= 1499 or max_mz - min_mz >= 99990:
        continue
        
    comb_img_features = []
    #loop through each individual image in folder 
    for f in files:
        if f.endswith(".png"):
            img = cv.imread(f)
            #crop individual image to look at region of feature A
            single_img_crop = img[min_mz:max_mz,min_rt:max_rt]

            #perform watershed on ROI in individual image
            gray = cv.cvtColor(single_img_crop, cv.COLOR_BGR2GRAY)
            gray_inv = np.invert(gray)
            kernel = np.ones((3,3),np.uint8)
            new_grad = cv.morphologyEx(gray_inv,cv.MORPH_GRADIENT, kernel, iterations =1)

            ret, new_markers = cv.connectedComponents(new_grad)
            new_markers = new_markers+1
            new_markers = cv.watershed(single_img_crop,new_markers)

            #for every feature in individual image that is in ROI
            #print(len(regionprops(new_markers)))
            #print(regionprops(new_markers)[0].bbox)
            single_img_feature = []
            
            for region in regionprops(new_markers):
                i_min_mz, i_min_rt, i_max_mz, i_max_rt = region.bbox
                i_med_mz = (i_min_mz+i_max_mz)/2
                i_med_rt = (i_min_rt+i_min_rt)/2
                mz = round(150 + min_mz*.001 + (i_med_mz)*.001,4)
                rt = round(min_rt*.02 + (i_med_rt)*.02,2)
                i = single_img_crop[int(i_med_mz),int(i_med_rt)][0]
                if i != 0:
                    single_img_feature.append([mz,rt,i])
                
            
            comb_img_features.append(single_img_feature)   
    feature_list[c] = [comb_img_features]
    c+=1
    if c == 10:
        break
    

In [ ]:
#individual sample featurization - V1

from pyteomics import mzxml, auxiliary
import matplotlib.patches as mpatches
import os
from statistics import median
from tqdm import tqdm_notebook as tqdm
import numpy as np

directory = "/home/rcardiff/ryan/ryan/meyer_raw_data/C18_neg/CL/"
os.chdir(directory)
files = os.listdir(directory)

feature_list = [{} for r in regionprops(markers)]

#for each feature in combined image:
regions = regionprops(markers)
for r in tqdm(regions):
    min_mz, min_rt, max_mz, max_rt = r.bbox
    #change x,y values to mz values
    if max_rt - min_rt >= 1499 or max_mz - min_mz >= 99990:
        continue
    min_mz = round(150 + min_mz*.001,4)
    min_rt = round(min_rt*.02,2)
    max_mz = round(150 + max_mz*.001,4)
    max_rt = round(max_rt*.02,2)    
    
    sample_list = []
    #for each sample:
    for f in files:
        if f.endswith(".mzXML"):
            sample_val, sample_mz, sample_rt, sample_i = [], [], [], []
            a = mzxml.read(f) 
            #for every spectra in sample:
            for spectra in a:
                spectra_rt = spectra.get('retentionTime')
                if spectra_rt >= min_rt and spectra_rt <= max_rt: #if rt in range
                    mz = spectra.get('m/z array')
                    intensity = spectra.get('intensity array')

                    for m,i in zip(mz,intensity):
                        if m >= min_mz and m <= max_mz: #if mz in range
                            sample_mz.append(m)
                            sample_rt.append(spectra_rt)
                            sample_i.append(i)
            
            if sample_mz: #if there is a spectra in the sample in the right range
                sample_min_mz, sample_max_mz, sample_med_mz = min(sample_mz), max(sample_mz), median(sample_mz)
                sample_min_rt, sample_max_rt, sample_med_rt = min(sample_rt), max(sample_rt), median(sample_rt)
                sample_tot_i, sample_med_i = sum(sample_i), median(sample_i)
                sample_val = [sample_min_mz, sample_max_mz, sample_med_mz, sample_min_rt, 
                           sample_max_rt, sample_med_rt, sample_tot_i, sample_med_i] #compute values for sample
    
            sample_list.append(sample_val) #list of each sample's values
        
            #print(sample_list)
            
    feature_list[regions.index(r)] = {"Feature": [min_mz,min_rt,max_mz,max_rt], "CL1":sample_list[0],
                                      "CL2": sample_list[1], "CL3":sample_list[2],"CL5":sample_list[3],
                                      "CL6":sample_list[4],"CL7": sample_list[5],"CL8":sample_list[6],
                                      "CL9":sample_list[7],"CL10":sample_list[8],"CL11":sample_list[9],
                                      "CL14":sample_list[10],"CL16":sample_list[11]}
    #if regions.index(r) > 10:
     #   break

In [5]:
#individual sample featurization - V2

from pyteomics import mzxml, auxiliary
import matplotlib.patches as mpatches
import os
from statistics import median
from tqdm import tqdm_notebook as tqdm
import numpy as np

feature_data = [x.bbox for x in regionprops(markers)]
feature_list = [{} for x in regionprops(markers)]

min_mzs = np.array([x[0] for x in feature_data])
min_rts = np.array([x[1] for x in feature_data])
max_mzs = np.array([x[2] for x in feature_data])
max_rts = np.array([x[3] for x in feature_data])

min_mzs = np.round(150.0 + (min_mzs * .001), decimals = 4)
min_rts = np.round(min_rts * .02, decimals = 4)
max_mzs = np.round(150.0 + (max_mzs * .001), decimals = 4)
max_rts = np.round(max_rts * .02, decimals = 4)


num_features = len(min_mzs)

directory = "/home/rcardiff/ryan/ryan/meyer_raw_data/C18_neg/CL/"
in_files = [os.path.join(directory, x) for x in os.listdir(directory) if x.endswith('.mzXML')]
feature_list = [{} for r in regionprops(markers)]

running_feature_list_mz = [[[] for f in in_files] for r in regionprops(markers)]
running_feature_list_rt = [[[] for f in in_files] for r in regionprops(markers)]
running_feature_list_i = [[[] for f in in_files] for r in regionprops(markers)]



for fname in in_files:

    
    break


In [142]:
print(len(feature_list))
print(len(feature_data))#len(running_feature_list[0])
#print(in_files.index(fname))
#running_feature_list[feature][file] = rfl[x][y]

196678
196676


In [6]:
file_index = in_files.index(fname)
f_data = mzxml.read(fname)

all_data =[(x['m/z array'], x['intensity array'], [float(x['retentionTime']) for y in x['m/z array']]) for x in f_data]


mzs = np.concatenate([x[0] for x in all_data])
intensities = np.concatenate([x[1] for x in all_data])
rts = np.concatenate([x[2] for x in all_data])

In [116]:
print(len(min_mzs))

196676


In [7]:
c = 0
for i_mz, mz in tqdm(enumerate(mzs)):
    c+=1
    rt = rts[i_mz]
    intensity = intensities[i_mz]
    
    minSub = mz - min_mzs
    minSub = (minSub > 0) * minSub
    maxSub = max_mzs - mz
    maxSub = (maxSub > 0) * maxSub
    mzCheck = np.multiply(minSub, maxSub)
    
    if any(mzCheck):
        minSub = rt - min_rts
        minSub = (minSub > 0) * minSub
        maxSub = max_rts - rt
        maxSub = (maxSub > 0) * maxSub
        rtCheck = np.multiply(minSub, maxSub)
        allCheck = np.multiply(mzCheck, rtCheck)
    
        if any(allCheck):
            for index in np.nonzero(allCheck)[0]:
                running_feature_list_mz[index][file_index].append(mz)
                running_feature_list_rt[index][file_index].append(rt)
                running_feature_list_i[index][file_index].append(intensity)
        
        if c >= 10000:
            break

In [123]:
for i,ft in enumerate(running_feature_list_mz):
    min_mz, min_rt, max_mz, max_rt = min_mzs[i],min_rts[i], max_mzs[i], max_rts[i]
    break

In [137]:
print(len(running_feature_list_mz))
print(len(min_mzs))

196678
196676


In [17]:
from statistics import mean,median

for i,ft in tqdm(enumerate(running_feature_list_mz)): #for every feature
    min_mz, min_rt, max_mz, max_rt = min_mzs[i],min_rts[i], max_mzs[i], max_rts[i]
    if max_rt - min_rt >= 1499 or max_mz - min_mz >= 99990:
        continue
    sample_list = []
    
    for j,file in enumerate(ft): #for each file
        sample_val = []
        if file:
            sample_min_mz, sample_max_mz, sample_med_mz = min(file), max(file), median(file)
            rt_list = running_feature_list_rt[i][j]
            i_list = running_feature_list_i[i][j]
            sample_min_rt, sample_max_rt, sample_med_rt = min(rt_list), max(rt_list), median(rt_list)
            sample_tot_i, sample_mean_i = sum(i_list), mean(i_list)
        
            sample_val = [sample_min_mz, sample_max_mz, sample_med_mz, sample_min_rt, 
                        sample_max_rt, sample_med_rt, sample_tot_i, sample_mean_i] #compute values for sample
    
        sample_list.append(sample_val)
        
    feature_list[i] = {"Feature ID": "FT"+str(i), "FT med mz":median([min_mz, max_mz]), "FT min mz": min_mz, "FT max mz": max_mz,
                                       "FT med rt": median([min_rt,max_rt]), "FT min rt": min_rt, "FT max rt": max_rt, 
                                       "CL1":sample_list[0],
                                      "CL2": sample_list[1], "CL3":sample_list[2],"CL5":sample_list[3],
                                      "CL6":sample_list[4],"CL7": sample_list[5],"CL8":sample_list[6],
                                      "CL9":sample_list[7],"CL10":sample_list[8],"CL11":sample_list[9],
                                      "CL14":sample_list[10],"CL16":sample_list[11]}
    

In [150]:
import csv
directory = "/home/rcardiff/high_res_test/"
os.chdir(directory)

keys = feature_list[2].keys()
#keys = feature_list[0].keys()
#keys = ["Feature", "CL1", "CL2", "CL3","CL5","CL6","CL7","CL8","CL9","CL10","CL11","CL14","CL16"]

with open('feature_full_NEWTEST.csv', 'w') as f:
    dict_writer = csv.DictWriter(f, keys)
    dict_writer.writeheader()
    dict_writer.writerows(feature_list[2:])

In [30]:
import pickle

#Write to pickle
filename = 'pickle'
outfile = open(filename, 'wb')
pickle.dump(feature_list,outfile)
outfile.close()

#Open pickle
#infile = open(filename,'rb')
#new_dict = pickle.load(infile)
#infile.close()

#Compress
import bz2
sfile = bz2.BZ2File('smallfile','w')
pickle.dump(feature_list,sfile)

In [28]:
import csv
directory = "/home/rcardiff/high_res_test/"
os.chdir(directory)

summary = {"Feature ID", "FT med mz", "FT min mz", "FT max mz", "FT med rt", "FT min rt", "FT max rt"}
#subList = { key:value for key,value in feature_list[2].items() if key in summary}
#print(subList)

with open('summary_test.csv', 'w') as f:
    dict_writer = csv.DictWriter(f, summary)
    dict_writer.writeheader()
    subList = ( { key:value for key,value in x.items() if key in summary} for x in feature_list[2:] )
    dict_writer.writerows(subList)

TypeError: '<' not supported between instances of 'dict' and 'dict'

[{"feature": [min_mz,min_rt,max_mz,max_rt],"CL1":[min_mz,max_mz,med_mz,min_rt,max_rt,med_rt,tot_i,med_i], 
  "CL2": [] }, {"feature": ...}, ...] 

-list of dict of list

-improve run time!!!
-improve formatting of csv... do we want 4+(12x8) columns? vs. 1+(12) but pretty unintelligible 
    -change by making the value for each dict a list of dicts:
    {"Feature": {"min_mz": min_mz,...},"CL1": {"min_mz": sample_list[0][0], "max_mz": sample_list[0][1]...}, "CL2":{...},...}
    -How to write this to CSV??
    
-thinking about beginning to group very like features

In [50]:
from pyteomics import mzxml, auxiliary
import matplotlib.patches as mpatches
import os
from statistics import median
from tqdm import tqdm_notebook as tqdm

directory = "/home/rcardiff/ryan/ryan/meyer_raw_data/C18_neg/CL/"
os.chdir(directory)
files = os.listdir(directory)

feature_list = [{} for r in regionprops(markers)]

#for each feature in combined image:
regions = regionprops(markers)
for r in tqdm(regions):
    min_mz, min_rt, max_mz, max_rt = r.bbox
    #change x,y values to mz values
    if max_rt - min_rt >= 1499 or max_mz - min_mz >= 99990:
        continue
    min_mz = round(150 + min_mz*.001,4)
    min_rt = round(min_rt*.02,2)
    max_mz = round(150 + max_mz*.001,4)
    max_rt = round(max_rt*.02,2)    
    
    sample_list = []
    #for each sample:
    for f in files:
        if f.endswith(".mzXML"):
            sample_val, sample_mz, sample_rt, sample_i = [], [], [], []
            a = mzxml.read(f) 
            #for every spectra in sample:
            for spectra in a:
                spectra_rt = spectra.get('retentionTime')
                if spectra_rt >= min_rt and spectra_rt <= max_rt: #if rt in range
                    mz = spectra.get('m/z array')
                    intensity = spectra.get('intensity array')

                    for m,i in zip(mz,intensity):
                        if m >= min_mz and m <= max_mz: #if mz in range
                            sample_mz.append(m)
                            sample_rt.append(spectra_rt)
                            sample_i.append(i)
            
            if sample_mz: #if there is a spectra in the sample in the right range
                sample_min_mz, sample_max_mz, sample_med_mz = min(sample_mz), max(sample_mz), median(sample_mz)
                sample_min_rt, sample_max_rt, sample_med_rt = min(sample_rt), max(sample_rt), median(sample_rt)
                sample_tot_i, sample_med_i = sum(sample_i), median(sample_i)
                sample_val = [sample_min_mz, sample_max_mz, sample_med_mz, sample_min_rt, 
                           sample_max_rt, sample_med_rt, sample_tot_i, sample_med_i] #compute values for sample
            else:
                sample_val = [[],[],[],[],[],[],[],[]]
            sample_list.append(sample_val) #list of each sample's values
        
    #feature_list[regions.index(r)] = { 
     #                                 "Feature": {"min_mz":min_mz,"min_rt":min_rt,"max_mz":max_mz,"max_rt":max_rt}, 
      #                                "CL1":{"min_mz":sample_list[0][0],"max_mz":sample_list[0][1],"med_mz":sample_list[0][2],"min_rt":sample_list[0][3],"max_rt":sample_list[0][4],"med_rt":sample_list[0][5], "tot_i":sample_list[0][6], "med_i":sample_list[0][7]},
       #                               "CL2": {"min_mz":sample_list[1][0],"max_mz":sample_list[1][1],"med_mz":sample_list[1][2],"min_rt":sample_list[1][3],"max_rt":sample_list[0][4],"med_rt":sample_list[1][5], "tot_i":sample_list[1][6], "med_i":sample_list[1][7]},
        #                              "CL3":{"min_mz":sample_list[2][0],"max_mz":sample_list[2][1],"med_mz":sample_list[2][2],"min_rt":sample_list[2][3],"max_rt":sample_list[2][4],"med_rt":sample_list[2][5], "tot_i":sample_list[2][6], "med_i":sample_list[2][7]},
         #                             "CL5":{"min_mz":sample_list[3][0],"max_mz":sample_list[3][1],"med_mz":sample_list[3][2],"min_rt":sample_list[3][3],"max_rt":sample_list[3][4],"med_rt":sample_list[3][5], "tot_i":sample_list[3][6], "med_i":sample_list[3][7]},
          #                            "CL6":{"min_mz":sample_list[4][0],"max_mz":sample_list[4][1],"med_mz":sample_list[4][2],"min_rt":sample_list[4][3],"max_rt":sample_list[4][4],"med_rt":sample_list[4][5], "tot_i":sample_list[4][6], "med_i":sample_list[4][7]},
           #                           "CL7": {"min_mz":sample_list[5][0],"max_mz":sample_list[5][1],"med_mz":sample_list[5][2],"min_rt":sample_list[5][3],"max_rt":sample_list[5][4],"med_rt":sample_list[5][5], "tot_i":sample_list[5][6], "med_i":sample_list[5][7]},
            #                          "CL8":{"min_mz":sample_list[6][0],"max_mz":sample_list[6][1],"med_mz":sample_list[6][2],"min_rt":sample_list[6][3],"max_rt":sample_list[6][4],"med_rt":sample_list[6][5], "tot_i":sample_list[6][6], "med_i":sample_list[6][7]},
             #                         "CL9":{"min_mz":sample_list[7][0],"max_mz":sample_list[7][1],"med_mz":sample_list[7][2],"min_rt":sample_list[7][3],"max_rt":sample_list[7][4],"med_rt":sample_list[7][5], "tot_i":sample_list[7][6], "med_i":sample_list[7][7]},
              #                        "CL10":{"min_mz":sample_list[8][0],"max_mz":sample_list[8][1],"med_mz":sample_list[8][2],"min_rt":sample_list[8][3],"max_rt":sample_list[8][4],"med_rt":sample_list[8][5], "tot_i":sample_list[8][6], "med_i":sample_list[8][7]},
               #                       "CL11":{"min_mz":sample_list[9][0],"max_mz":sample_list[9][1],"med_mz":sample_list[9][2],"min_rt":sample_list[9][3],"max_rt":sample_list[9][4],"med_rt":sample_list[9][5], "tot_i":sample_list[9][6], "med_i":sample_list[9][7]},
                #                      "CL14":{"min_mz":sample_list[10][0],"max_mz":sample_list[10][1],"med_mz":sample_list[10][2],"min_rt":sample_list[10][3],"max_rt":sample_list[10][4],"med_rt":sample_list[10][5], "tot_i":sample_list[10][6], "med_i":sample_list[10][7]},
                 #                     "CL16":{"min_mz":sample_list[11][0],"max_mz":sample_list[11][1],"med_mz":sample_list[11][2],"min_rt":sample_list[11][3],"max_rt":sample_list[11][4],"med_rt":sample_list[11][5], "tot_i":sample_list[11][6], "med_i":sample_list[11][7]}   
                  #                    }
    feature_list[regions.index(r)] = {"Feature min_mz":min_mz,"Feature min_rt":min_rt,"Feature max_mz":max_mz,"Feature max_rt":max_rt, 
                                      "CL1 min_mz":sample_list[0][0],"CL1 max_mz":sample_list[0][1],"CL1 med_mz":sample_list[0][2],"CL1 min_rt":sample_list[0][3],"CL1 max_rt":sample_list[0][4],"CL1 med_rt":sample_list[0][5], "CL1 tot_i":sample_list[0][6], "CL1 med_i":sample_list[0][7],
                                      "CL2 min_mz":sample_list[1][0],"CL2 max_mz":sample_list[1][1],"CL2 med_mz":sample_list[1][2],"CL2 min_rt":sample_list[1][3],"CL2 max_rt":sample_list[0][4],"CL2 med_rt":sample_list[1][5], "CL2 tot_i":sample_list[1][6], "CL2 med_i":sample_list[1][7],
                                      "CL3 min_mz":sample_list[2][0],"CL3 max_mz":sample_list[2][1],"CL3 med_mz":sample_list[2][2],"CL3 min_rt":sample_list[2][3],"CL3 max_rt":sample_list[2][4],"CL3 med_rt":sample_list[2][5], "CL3 tot_i":sample_list[2][6], "CL3 med_i":sample_list[2][7],
                                      "CL5 min_mz":sample_list[3][0],"CL5 max_mz":sample_list[3][1],"CL5 med_mz":sample_list[3][2],"CL5 min_rt":sample_list[3][3],"CL5 max_rt":sample_list[3][4],"CL5 med_rt":sample_list[3][5], "CL5 tot_i":sample_list[3][6], "CL5 med_i":sample_list[3][7],
                                      "CL6 min_mz":sample_list[4][0],"CL6 max_mz":sample_list[4][1],"CL6 med_mz":sample_list[4][2],"CL6 min_rt":sample_list[4][3],"CL6 max_rt":sample_list[4][4],"CL6 med_rt":sample_list[4][5], "CL6 tot_i":sample_list[4][6], "CL6 med_i":sample_list[4][7],
                                      "CL7 min_mz":sample_list[5][0],"CL7 max_mz":sample_list[5][1],"CL7 med_mz":sample_list[5][2],"CL7 min_rt":sample_list[5][3],"CL7 max_rt":sample_list[5][4],"CL7 med_rt":sample_list[5][5], "CL7 tot_i":sample_list[5][6], "CL7 med_i":sample_list[5][7],
                                      "CL8 min_mz":sample_list[6][0],"CL8 max_mz":sample_list[6][1],"CL8 med_mz":sample_list[6][2],"CL8 min_rt":sample_list[6][3],"CL8 max_rt":sample_list[6][4],"CL8 med_rt":sample_list[6][5], "CL8 tot_i":sample_list[6][6], "CL8 med_i":sample_list[6][7],
                                      "CL9 min_mz":sample_list[7][0],"CL9 max_mz":sample_list[7][1],"CL9 med_mz":sample_list[7][2],"CL9 min_rt":sample_list[7][3],"CL9 max_rt":sample_list[7][4],"CL9 med_rt":sample_list[7][5], "CL9 tot_i":sample_list[7][6], "CL9 med_i":sample_list[7][7],
                                      "CL10 min_mz":sample_list[8][0],"CL10 max_mz":sample_list[8][1],"CL10 med_mz":sample_list[8][2],"CL10 min_rt":sample_list[8][3],"CL10 max_rt":sample_list[8][4],"CL10 med_rt":sample_list[8][5], "CL10 tot_i":sample_list[8][6], "CL10 med_i":sample_list[8][7],
                                      "CL11 min_mz":sample_list[9][0],"CL11 max_mz":sample_list[9][1],"CL11 med_mz":sample_list[9][2],"CL11 min_rt":sample_list[9][3],"CL11 max_rt":sample_list[9][4],"CL11 med_rt":sample_list[9][5], "CL11 tot_i":sample_list[9][6], "CL11 med_i":sample_list[9][7],
                                      "CL14 min_mz":sample_list[10][0],"CL14 max_mz":sample_list[10][1],"CL14 med_mz":sample_list[10][2],"CL14 min_rt":sample_list[10][3],"CL14 max_rt":sample_list[10][4],"CL14 med_rt":sample_list[10][5], "CL14 tot_i":sample_list[10][6], "CL14 med_i":sample_list[10][7],
                                      "CL16 min_mz":sample_list[11][0],"CL16 max_mz":sample_list[11][1],"CL16 med_mz":sample_list[11][2],"CL16 min_rt":sample_list[11][3],"CL16 max_rt":sample_list[11][4],"CL16 med_rt":sample_list[11][5], "CL16 tot_i":sample_list[11][6], "CL16 med_i":sample_list[11][7]   
                                      }
    if regions.index(r) > 10:
        break

In [ ]:
keys = ["Feature", "CL1", "CL2", "CL3","CL5","CL6","CL7","CL8","CL9","CL10","CL11","CL14","CL16"]
sample_list = [[], [], [], [], [], [], [], [150.0049052782952, 150.0049052782952, 150.0049052782952, 16.364383333333333, 16.364383333333333, 16.364383333333333, 200.65719604492188, 200.65719604492188], [], [], [], []]
print(sample_list[7][1])
keys= (sample_list[0][i] for i in range(2))
#dict = {keys[i]:sample_list[0][i] for i in range (7)}
print(keys)

In [47]:
import csv
import itertools
import sys
directory = "/home/rcardiff/high_res_test/"
os.chdir(directory)


feat = feature_list[2]
fields = ['Feature', 'min_mz', 'min_rt','max_mz','max_rt','med_mz',"med_rt","tot_i","med_i","CL1", "CL2", "CL3","CL5","CL6","CL7","CL8","CL9","CL10","CL11","CL14","CL16" ]

with open("test_output.csv", "w") as f:
    w = csv.DictWriter( f, fields )
    w.writeheader()
    for key,val in sorted(feat.items()):
        row = {'Feature': key}
        row.update(val)
        w.writerow(row)